In [1]:
import os
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import transforms, datasets, models, utils
import pretrainedmodels
from tqdm import tqdm

In [2]:
data_dir = './Images/'
use_cuda = True
device_gpu = torch.device("cuda")
device_cpu = torch.device("cpu")
batch_size = 64

In [3]:
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transform_test = transforms.Compose([
    transforms.RandomResizedCrop(299),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transform_train_299 = transforms.Compose([
    transforms.RandomResizedCrop(299),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transform_test_299 = transforms.Compose([
    transforms.RandomResizedCrop(299),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [4]:
train_dataset = datasets.ImageFolder(data_dir, transform=transform_train)
train_dataset_299 = datasets.ImageFolder(data_dir, transform=transform_train_299)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False, drop_last=False)
train_dataloader_299 = torch.utils.data.DataLoader(train_dataset_299, batch_size=batch_size, shuffle=False, drop_last=False)

In [5]:
def save_features(model_name, dataloader, device, phase):
    if model_name.startswith("densenet"):
        model = models.__dict__[model_name](pretrained=True)
    else:
        model = pretrainedmodels.__dict__[model_name](pretrained='imagenet')
    
    model.to(device)
    model.eval()
    
    features_model_path = "model_stanford/features_%s_%s.npy" % (model_name, phase)
    ytargets_model_path = "model_stanford/ytargets_%s.npy" % (phase)
    
    features = []
    ytargets = []
    
    with torch.no_grad():
        for idx, (data, labels) in tqdm(enumerate(dataloader, 0)):
            data, labels = data.to(device), labels.to(device)
            
            if model_name.startswith('vgg'):
                x = model.features(data)
            elif model_name.startswith('inception') or model_name.startswith('res') or model_name.startswith('densenet'):
                x = model.features(data)
                x = F.avg_pool2d(x, kernel_size=x.size(-1), stride=1)
                x = x.view(x.size(0), -1)
            
            features.append(x.cpu().numpy())
            ytargets.append(labels.cpu().numpy())
    
    features = np.concatenate(features, axis=0)
    ytargets = np.concatenate(ytargets, axis=0)
    print(features.shape, ytargets.shape)
    
    np.save(features_model_path, features)
    if not os.path.exists(ytargets_model_path):
        np.save(ytargets_model_path, ytargets)
    
    del model
    torch.cuda.empty_cache()

In [ ]:
save_features("resnet152", train_dataloader, device_gpu, 'train')
save_features("vgg19_bn", train_dataloader, device_gpu, 'train')
save_features("inceptionv3", train_dataloader_299, device_gpu, 'train')
save_features("densenet201", train_dataloader, device_gpu, 'train')